## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [2]:
class myModel:
    def __init__(self):
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[28*28, 100],
                                                         minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[100, 10],
                                                         minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
    def __call__(self, x):
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [3]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [4]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3515916 ; accuracy 0.1055
epoch 1 : loss 2.3465188 ; accuracy 0.1068
epoch 2 : loss 2.3414927 ; accuracy 0.10845
epoch 3 : loss 2.336513 ; accuracy 0.11001667
epoch 4 : loss 2.3315775 ; accuracy 0.11155
epoch 5 : loss 2.326685 ; accuracy 0.113866664
epoch 6 : loss 2.3218336 ; accuracy 0.11573333
epoch 7 : loss 2.3170218 ; accuracy 0.11775
epoch 8 : loss 2.312249 ; accuracy 0.11998333
epoch 9 : loss 2.307513 ; accuracy 0.12211667
epoch 10 : loss 2.3028133 ; accuracy 0.124516666
epoch 11 : loss 2.2981484 ; accuracy 0.1269
epoch 12 : loss 2.293517 ; accuracy 0.12961666
epoch 13 : loss 2.2889185 ; accuracy 0.13218333
epoch 14 : loss 2.284351 ; accuracy 0.13483334
epoch 15 : loss 2.279814 ; accuracy 0.13766667
epoch 16 : loss 2.2753067 ; accuracy 0.14066666
epoch 17 : loss 2.270828 ; accuracy 0.14391667
epoch 18 : loss 2.2663767 ; accuracy 0.14741667
epoch 19 : loss 2.2619524 ; accuracy 0.15088333
epoch 20 : loss 2.2575536 ; accuracy 0.15416667
epoch 21 : loss 2.25318 ; acc